Code adapted from NVIDIA OpenACC example: laplace2d-kernels.c

Copyright (c) 2012, NVIDIA CORPORATION. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:
 * Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
 * Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
 * Neither the name of NVIDIA CORPORATION nor the names of its
   contributors may be used to endorse or promote products derived
   from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT OWNER OR
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [2]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
%load_ext snakeviz

In [4]:
import numpy as np
import time

In [5]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:n-1, 1:m-1] = 0.25 * (A[1:n-1, 2:] + A[1:n-1, :m-2] + A[2:, 1:m-1] + A[:m-2, 1:n-1])      
        diff = np.abs(Anew - A)
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A

In [6]:
%lprun -f np_laplace np_laplace(1024,1024,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 44.43885779380798 s



In [52]:
n_loops = 10
N = [2 ** val for val in range(n_loops)]

In [ ]:
np_res = []
for n in N:
    res = %timeit -o -n 3 np_laplace(n, n)
    np_res.append((n, res))

/home/schowell/data/myPrograms/anaconda3/envs/cuda/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in true_divide


Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 0.00012683868408203125 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 5.435943603515625e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.9591064453125e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.9591064453125e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.8160552978515625e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.863739013671875e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.8160552978515625e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.8160552978515625e-05 s

Jacobi relaxation Calculation: 1 x 1 mesh

    0, 0.000000

 total: 4.792213439941406e-05 s

The slowest run took 9.42 times longer than the fastest. This could mean that an intermediate result is being cached.
3 loops, best of 3: 99.4 µs per loop
Jacobi r

# Adding numba jit decorator

In [ ]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:n-1, 1:m-1] = 0.25 * (A[1:n-1, 2:] + A[1:n-1, :m-2] + A[2:, 1:m-1] + A[:m-2, 1:n-1])      
        diff = np.abs(Anew - A)
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A

# Implementing `guvectorize`

In [ ]:
from numba import guvectorize

In [ ]:
@guvectorize(['float32[:,:], float32[:,:]',
              'float64[:,:], float64[:,:]'],
             '(n,m)->(n,m)', nopython=True)
def update_a(A, Anew):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

In [ ]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:n-1, 1:m-1] = 0.25 * (A[1:n-1, 2:] + A[1:n-1, :m-2] + A[2:, 1:m-1] + A[:m-2, 1:n-1])      
        diff = np.abs(Anew - A)
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    print(" total: {} s\n".format(toc))
    
    return A